### Notebook used in the assignment of building damage data to grids from municipalities


In [1]:
import pandas as pd

In [2]:
# reading in file with building damage data and adm3 in grid
adm3_perc_ingrid = pd.read_csv("C:/Users/pauni/Desktop/Work/OCHA/Global Typhoon/Adm3_Perc_inGrid_Weight_Matrix.csv")
build_dmg_data = pd.read_csv("G:/Shared drives/Predictive Analytics/Collaborations/ISI/data/analysis/01_baseline_model/input/combined_input_data.csv")
adm3_perc_ingrid.shape


(9879, 5)

In [3]:
build_dmg_data.shape

(25843, 38)

In [4]:
build_dmg_data.columns

Index(['Mun_Code', 'typhoon', 'HAZ_rainfall_Total', 'HAZ_rainfall_max_6h',
       'HAZ_rainfall_max_24h', 'HAZ_v_max', 'HAZ_dis_track_min',
       'GEN_landslide_per', 'GEN_stormsurge_per', 'GEN_Bu_p_inSSA',
       'GEN_Bu_p_LS', 'GEN_Red_per_LSbldg', 'GEN_Or_per_LSblg',
       'GEN_Yel_per_LSSAb', 'GEN_RED_per_SSAbldg', 'GEN_OR_per_SSAbldg',
       'GEN_Yellow_per_LSbl', 'TOP_mean_slope', 'TOP_mean_elevation_m',
       'TOP_ruggedness_stdev', 'TOP_mean_ruggedness', 'TOP_slope_stdev',
       'VUL_poverty_perc', 'GEN_with_coast', 'GEN_coast_length',
       'VUL_Housing_Units', 'VUL_StrongRoof_StrongWall',
       'VUL_StrongRoof_LightWall', 'VUL_StrongRoof_SalvageWall',
       'VUL_LightRoof_StrongWall', 'VUL_LightRoof_LightWall',
       'VUL_LightRoof_SalvageWall', 'VUL_SalvagedRoof_StrongWall',
       'VUL_SalvagedRoof_LightWall', 'VUL_SalvagedRoof_SalvageWall',
       'VUL_vulnerable_groups', 'VUL_pantawid_pamilya_beneficiary',
       'DAM_perc_dmg'],
      dtype='object')

In [5]:
# combining building damage data and grid information
merged_df = adm3_perc_ingrid.merge(build_dmg_data[["Mun_Code", "typhoon", "DAM_perc_dmg"]], left_on="ADM3_PCODE", right_on="Mun_Code", how="left")
merged_df["DAM_perc_dmg"].describe()
merged_df

,id,Centroid,ADM3_PCODE,ADM3_EN,Grid Completeness,Mun_Code,typhoon,DAM_perc_dmg
0,6755,118.3E_9.7N,PH175301000,Aborlan,0.001601,PH175301000,bopha2012,NaN
1,6755,118.3E_9.7N,PH175301000,Aborlan,0.001601,PH175301000,haiyan2013,NaN
2,6755,118.3E_9.7N,PH175301000,Aborlan,0.001601,PH175301000,jangmi2014,NaN
3,6756,118.3E_9.6N,PH175301000,Aborlan,0.241804,PH175301000,bopha2012,NaN
4,6756,118.3E_9.6N,PH175301000,Aborlan,0.241804,PH175301000,haiyan2013,NaN
...,...,...,...,...,...,...,...,...
147025,17692,124.9E_11.6N,PH086023000,Zumarraga,0.070106,PH086023000,rammasun2014,NaN
147026,17692,124.9E_11.6N,PH086023000,Zumarraga,0.070106,PH086023000,sarika2016,NaN
147027,17692,124.9E_11.6N,PH086023000,Zumarraga,0.070106,PH086023000,tokage2016,NaN
147028,17692,124.9E_11.6N,PH086023000,Zumarraga,0.070106,PH086023000,vongfong2020,NaN


In [6]:
# computing % in each grid
merged_df["DAM_perc_dmg_Grid"] = merged_df["Grid Completeness"] * merged_df["DAM_perc_dmg"]
merged_df

,id,Centroid,ADM3_PCODE,ADM3_EN,Grid Completeness,Mun_Code,typhoon,DAM_perc_dmg,DAM_perc_dmg_Grid
0,6755,118.3E_9.7N,PH175301000,Aborlan,0.001601,PH175301000,bopha2012,NaN,NaN
1,6755,118.3E_9.7N,PH175301000,Aborlan,0.001601,PH175301000,haiyan2013,NaN,NaN
2,6755,118.3E_9.7N,PH175301000,Aborlan,0.001601,PH175301000,jangmi2014,NaN,NaN
3,6756,118.3E_9.6N,PH175301000,Aborlan,0.241804,PH175301000,bopha2012,NaN,NaN
4,6756,118.3E_9.6N,PH175301000,Aborlan,0.241804,PH175301000,haiyan2013,NaN,NaN
...,...,...,...,...,...,...,...,...,...
147025,17692,124.9E_11.6N,PH086023000,Zumarraga,0.070106,PH086023000,rammasun2014,NaN,NaN
147026,17692,124.9E_11.6N,PH086023000,Zumarraga,0.070106,PH086023000,sarika2016,NaN,NaN
147027,17692,124.9E_11.6N,PH086023000,Zumarraga,0.070106,PH086023000,tokage2016,NaN,NaN
147028,17692,124.9E_11.6N,PH086023000,Zumarraga,0.070106,PH086023000,vongfong2020,NaN,NaN


In [7]:
# writing output to CSV file
merged_df.to_csv("C:/Users/pauni/Desktop/Work/OCHA/Global Typhoon/building_dmg_bygrid.csv", index=False)
